In [5]:
# Import relevant libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.optimize import minimize
# Suppress annoying warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Import + format dataset
path = '/Users/julienraffaud/Desktop/Machine Learning with Applications in Finance/48_Industry_Portfolios_daily.CSV'
data = pd.read_csv(path)
data[data.columns[0]] = pd.to_datetime(data[data.columns[0]].astype(str), errors='coerce')
data = data.rename(columns={ data.columns[0]: "Date" })
data = data.set_index('Date')
data = data.apply(pd.to_numeric, errors='coerce')
data = data.iloc[-1500:, :]/100

In [7]:
# number of assets
dim = 48
# sample size (250=1year, 125=6months, 63=3months)
tau = 205
# length of out-of-sample window in days
out = 80
# VaR threshold
var_thresh = 95
# number of cross-validation folds
k = 10
# number of lambda to test @ each fold
grid = 30

Regulariser penalty functions:

In [4]:
def MVP(x, cov):
    return np.linalg.multi_dot((x, cov, x.T))

def LASSO(x, cov, lmbd):
    return 10**4*(np.linalg.multi_dot((x, cov, x.T)) + lmbd*np.sum(abs(x)))

def RIDGE(x, cov, lmbd):
    return 10**4*(np.linalg.multi_dot((x, cov, x.T)) + lmbd*np.sum(abs(x)**2))

def W8LASS(x, cov, lmbd):
    gamma = 0.5
    indiv_weights = 1/(abs(x)**gamma)
    return 10**4*(np.linalg.multi_dot((x, cov, x.T)) + np.sum(np.dot(lmbd*indiv_weights, abs(x.T))))

def SCAD(x, cov, lmbd):
    a = 3.7
    variance = np.linalg.multi_dot((x, cov, x.T))
    x_mod = np.copy(x)
    x_mod[abs(x_mod)<=lmbd] = lmbd*abs(x_mod[abs(x_mod)<=lmbd])
    x_mod[abs(x_mod)>lmbd*a] = ((a+1)*lmbd**2)/2
    x_mod[(abs(x_mod) > lmbd ) & (abs(x_mod) <= a*lmbd)] = (-abs(x_mod[(abs(x_mod) > lmbd ) & (abs(x_mod) <= a*lmbd)])**2 + 2*a*lmbd*abs(x_mod[(abs(x_mod) > lmbd ) & (abs(x_mod) <= a*lmbd)]) - lmbd**2 )/(2*(a-1))
    return 10**4*(variance + np.sum(x_mod))

def Zhang(x, cov, lmbd):
    variance =  np.linalg.multi_dot((x, cov, x.T))
    eps = 0.005
    x_mod = np.copy(x)
    x_mod[abs(x_mod)>=eps] = eps
    reg = lmbd*np.sum(abs(x))
    return 10**4*(variance + reg)

def Lq(x, cov, lmbd):
    return 10**4*(np.linalg.multi_dot((x, cov, x.T)) + lmbd*np.sum(abs(x)**0.5))

def Log(x, cov, lmbd):
    psi = 0.01
    return 10**4*(np.linalg.multi_dot((x, cov, x.T)) + lmbd*np.sum(np.log((abs(x)+psi)/(psi))))

# defining linear constraint
cons = {'type':'eq', 'fun': lambda x: np.sum(x) - 1}

#define starting weights
x0 = np.ones((1, dim))*(1/dim)

Equally-weighted invariant portfolio (benchmark):

In [5]:
# Equal-weights portfolio backtest (benchmark)
bench_returns = []
for i in range(0, int((len(data) - tau)/out)):
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # equal weights
    mv_equal = (1/dim)*np.ones((1, dim)).T
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mv_equal.T, out_sample)
    bench_returns += out_returns.T.tolist()

In [6]:
# compute return variance
equal_variance = np.var(bench_returns)
# compute VaR at (1-var_thresh)% level
equal_var = np.percentile(bench_returns, 100-var_thresh)
# compute Sharpe ratio
equal_sharpe = np.mean( bench_returns )/np.sqrt(equal_variance)
print(equal_variance)
print(equal_var)
print(equal_sharpe)

7.53184639642265e-05
-0.01435875
0.06616718291145465


Minimum Variance Portfolio:

In [7]:
# Minimum variance portfolio backtest
mvp_returns = []
for i in range(0, int((len(data) - tau)/out)):
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # Estimated covariance matrix
    est_cov = np.cov(window.T)
    # Inverse of estimated covariance matrix
    cov_inv = np.linalg.inv( est_cov )
    # dim*1 vector of ones
    ones = np.ones((dim, 1))
    # First half of mvp weights formula
    a = np.linalg.inv( np.linalg.multi_dot(( ones.T, cov_inv, ones)) )
    # Second half of mvp weights formula
    b = np.dot( cov_inv, ones)
    # Minimum Variance Portfolio weights
    mvp = a*b
    # In-sample variance of the MVP
    var_in = np.linalg.multi_dot((mvp.T, est_cov, mvp))
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mvp.T, out_sample)
    mvp_returns += out_returns.T.tolist()

In [8]:
# compute MVP variance
mvp_variance = np.var(mvp_returns)
# compute MVP VaR at (1-var_thresh)% level
mvp_var = np.percentile(mvp_returns, 100-var_thresh)
# compute MVP Sharpe ratio
mvp_sharpe = np.mean( mvp_returns )/np.sqrt(mvp_variance)
print(mvp_variance)
print(mvp_var)
print(mvp_sharpe)

3.50194189340695e-05
-0.008965540384127037
0.15278747559358033


LASSO-regularised portfolio:

In [9]:
# LASSO-regularised portfolio backtest
# lambdas
lmbd = np.linspace(0, 3.*10**(-5), grid)
LASSO_returns = []
for i in range(0, int((len(data) - tau)/out)):
    print(i)
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # average out-of-sample variance associated with each lambda
    lmbd_variances = np.zeros((len(lmbd), 1))
    for fold in range(0, k):
        variances = []
        # sample values from in-sample data
        sample = np.random.choice(tau, out, replace=False)
        # remaining in-sample data
        mod_window = np.delete(window, sample, axis=0)
        # out-of-sample data
        outer = window[sample, :]
        # Estimated covariance matrix
        est_cov = np.cov(mod_window.T)
        ## CROSS-VALIDATION STEP
        for l in lmbd:
            # Portfolio weights
            mvp = minimize(LASSO, x0, (est_cov, l), constraints=cons).x
            # out-of-sample variance associated to each lambda
            var_out = np.var(np.dot(mvp.T, outer.T).T )
            # append variance
            variances.append( var_out )
        variances = np.array(variances)
        variances.shape = (len(lmbd), 1)
        # update each lambda's corresponding variance
        lmbd_variances += variances/k
    # index of lambda*
    star = lmbd_variances.tolist().index(min(lmbd_variances)) 
    # lambda*
    lambda_star = lmbd[lmbd_variances.tolist().index(min(lmbd_variances))]
    ## END OF CROSS VALIDATION STEP
    # estimated covariance matrix
    est_cov = np.cov(window.T)
    # Portfolio weights
    mvp = minimize(LASSO, x0, (est_cov, lambda_star), constraints=cons).x
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mvp.T, out_sample)
    LASSO_returns += out_returns.T.tolist()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [10]:
# compute LASSO variance
LASSO_variance = np.var(LASSO_returns)
# compute LASSO VaR at (1-var_thresh)% level
LASSO_var = np.percentile(LASSO_returns, 100-var_thresh)
# compute LASSO Sharpe ratio
LASSO_sharpe = np.mean( LASSO_returns )/np.sqrt(LASSO_variance)
print(LASSO_variance)
print(LASSO_var)
print(LASSO_sharpe)

3.284804329020193e-05
-0.008835405975458628
0.15662634425584376


Ridge-regularised portfolio:

In [11]:
# RIDGE-regularised portfolio backtest
# lambdas
lmbd = np.linspace(0, 13.5*10**(-5), grid)
RIDGE_returns = []
for i in range(0, int((len(data) - tau)/out)):
    print(i)
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # average out-of-sample variance associated with each lambda
    lmbd_variances = np.zeros((len(lmbd), 1))
    for fold in range(0, k):
        variances = []
        # sample values from in-sample data
        sample = np.random.choice(tau, out, replace=False)
        # remaining in-sample data
        mod_window = np.delete(window, sample, axis=0)
        # out-of-sample data
        outer = window[sample, :]
        # Estimated covariance matrix
        est_cov = np.cov(mod_window.T)
        ## CROSS-VALIDATION STEP
        for l in lmbd:
            # Portfolio weights
            mvp = minimize(RIDGE, x0, (est_cov, l), constraints=cons).x
            # out-of-sample variance associated to each lambda
            var_out = np.var(np.dot(mvp.T, outer.T).T )
            # append variance
            variances.append( var_out )
        variances = np.array(variances)
        variances.shape = (len(lmbd), 1)
        # update each lambda's corresponding variance
        lmbd_variances += variances/k
    # index of lambda*
    star = lmbd_variances.tolist().index(min(lmbd_variances)) 
    # lambda*
    lambda_star = lmbd[lmbd_variances.tolist().index(min(lmbd_variances))]
    ## END OF CROSS VALIDATION STEP
    # estimated covariance matrix
    est_cov = np.cov(window.T)
    # Portfolio weights
    mvp = minimize(RIDGE, x0, (est_cov, lambda_star), constraints=cons).x
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mvp.T, out_sample)
    RIDGE_returns += out_returns.T.tolist()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [12]:
# compute Ridge variance
RIDGE_variance = np.var(RIDGE_returns)
# Compute Ridge VaR at (1-var_thresh)% level
RIDGE_var = np.percentile(RIDGE_returns, 100-var_thresh)
# compute Ridge Sharpe ratio
RIDGE_sharpe = np.mean( RIDGE_returns )/np.sqrt(RIDGE_variance)
print(RIDGE_variance)
print(RIDGE_var)
print(RIDGE_sharpe)

3.240363128621679e-05
-0.008984323242248855
0.14491071562211344


Adapted Lasso (w8LASS) portfolio:

In [13]:
# w8LASS-regularised portfolio backtest
# lambdas
lmbd = np.linspace(0, 7.5*10**(-6), grid)
W8_returns = []
for i in range(0, int((len(data) - tau)/out)):
    print(i)
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # average out-of-sample variance associated with each lambda
    lmbd_variances = np.zeros((len(lmbd), 1))
    for fold in range(0, k):
        variances = []
        # sample values from in-sample data
        sample = np.random.choice(tau, out, replace=False)
        # remaining in-sample data
        mod_window = np.delete(window, sample, axis=0)
        # out-of-sample data
        outer = window[sample, :]
        # Estimated covariance matrix
        est_cov = np.cov(mod_window.T)
        ## CROSS-VALIDATION STEP
        for l in lmbd:
            # Portfolio weights
            mvp = minimize(W8LASS, x0, (est_cov, l), constraints=cons, tol=10**(-4)).x
            # out-of-sample variance associated to each lambda
            var_out = np.var(np.dot(mvp.T, outer.T).T )
            # append variance
            variances.append( var_out )
        variances = np.array(variances)
        variances.shape = (len(lmbd), 1)
        # update each lambda's corresponding variance
        lmbd_variances += variances/k
    # index of lambda*
    star = lmbd_variances.tolist().index(min(lmbd_variances)) 
    # lambda*
    lambda_star = lmbd[lmbd_variances.tolist().index(min(lmbd_variances))]
    ## END OF CROSS VALIDATION STEP
    # estimated covariance matrix
    est_cov = np.cov(window.T)
    # Portfolio weights
    mvp = minimize(W8LASS, x0, (est_cov, lambda_star), constraints=cons).x
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mvp.T, out_sample)
    W8_returns += out_returns.T.tolist()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [14]:
# compute W8 variance
W8_variance = np.var(W8_returns)
# compute W8 VaR at (1-var_thresh)% level
W8_var = np.percentile(W8_returns, 100-var_thresh)
# compute W8 Sharpe ratio
W8_sharpe = np.mean( W8_returns )/np.sqrt(W8_variance)
print(W8_variance)
print(W8_var)
print(W8_sharpe)

3.6828938416035383e-05
-0.009216850443639393
0.14620402944929478


SCAD-regularised portfolio:

In [15]:
# SCAD-regularised portfolio backtest
# lambdas
lmbd = np.linspace(0, 17*10**(-3.4), grid)
scad_returns = []
for i in range(0, int((len(data) - tau)/out)):
    print(i)
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # average out-of-sample variance associated with each lambda
    lmbd_variances = np.zeros((len(lmbd), 1))
    for fold in range(0, k):
        variances = []
        # sample values from in-sample data
        sample = np.random.choice(tau, out, replace=False)
        # remaining in-sample data
        mod_window = np.delete(window, sample, axis=0)
        # out-of-sample data
        outer = window[sample, :]
        # Estimated covariance matrix
        est_cov = np.cov(mod_window.T)
        ## CROSS-VALIDATION STEP
        for l in lmbd:
            # Portfolio weights
            mvp = minimize(SCAD, x0, (est_cov, l), constraints=cons, tol=10**(-4)).x
            # out-of-sample variance associated to each lambda
            var_out = np.var(np.dot(mvp.T, outer.T).T )
            # append variance
            variances.append( var_out )
        variances = np.array(variances)
        variances.shape = (len(lmbd), 1)
        # update each lambda's corresponding variance
        lmbd_variances += variances/k
    # index of lambda*
    star = lmbd_variances.tolist().index(min(lmbd_variances)) 
    # lambda*
    lambda_star = lmbd[lmbd_variances.tolist().index(min(lmbd_variances))]
    ## END OF CROSS VALIDATION STEP
    # estimated covariance matrix
    est_cov = np.cov(window.T)
    # Portfolio weights
    mvp = minimize(SCAD, x0, (est_cov, lambda_star), constraints=cons).x
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mvp.T, out_sample)
    scad_returns += out_returns.T.tolist()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [16]:
# compute SCAD variance
SCAD_variance = np.var(scad_returns)
# compute SCAD VaR at (1-var_thresh)% level
SCAD_var = np.percentile(scad_returns, 100-var_thresh)
# compute SCAD Sharpe ratio
SCAD_sharpe = np.mean( scad_returns )/np.sqrt(SCAD_variance)
print(SCAD_variance)
print(SCAD_var)
print(SCAD_sharpe)

3.457068964299519e-05
-0.008797461469111577
0.1467008668837796


Log-regularised portfolio:

In [17]:
# Log-regularised portfolio backtest
# lambdas
lmbd = np.linspace(0, 45*10**(-8), grid)
log_returns = []
for i in range(0, int((len(data) - tau)/out)):
    print(i)
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # average out-of-sample variance associated with each lambda
    lmbd_variances = np.zeros((len(lmbd), 1))
    for fold in range(0, k):
        variances = []
        # sample values from in-sample data
        sample = np.random.choice(tau, out, replace=False)
        # remaining in-sample data
        mod_window = np.delete(window, sample, axis=0)
        # out-of-sample data
        outer = window[sample, :]
        # Estimated covariance matrix
        est_cov = np.cov(mod_window.T)
        ## CROSS-VALIDATION STEP
        for l in lmbd:
            # Portfolio weights
            mvp = minimize(Log, x0, (est_cov, l), constraints=cons, tol=10**(-4)).x
            # out-of-sample variance associated to each lambda
            var_out = np.var(np.dot(mvp.T, outer.T).T )
            # append variance
            variances.append( var_out )
        variances = np.array(variances)
        variances.shape = (len(lmbd), 1)
        # update each lambda's corresponding variance
        lmbd_variances += variances/k
    # index of lambda*
    star = lmbd_variances.tolist().index(min(lmbd_variances)) 
    # lambda*
    lambda_star = lmbd[lmbd_variances.tolist().index(min(lmbd_variances))]
    ## END OF CROSS VALIDATION STEP
    # estimated covariance matrix
    est_cov = np.cov(window.T)
    # Portfolio weights
    mvp = minimize(Log, x0, (est_cov, lambda_star), constraints=cons).x
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mvp.T, out_sample)
    log_returns += out_returns.T.tolist()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [18]:
# compute log variance
log_variance = np.var(log_returns)
# compute log VaR at (1-var_thresh)% level
log_var = np.percentile(log_returns, 100-var_thresh)
# compute log Sharpe ratio
log_sharpe = np.mean( log_returns )/np.sqrt(log_variance)
print(log_variance)
print(log_var)
print(log_sharpe)

3.525748070382623e-05
-0.008884578086235809
0.1589493787023106


Lq-regularised portfolio:

In [19]:
# Lq-regularised portfolio backtest
# lambdas
lmbd = np.linspace(0, 48*10**(-7), grid)
lq_returns = []
for i in range(0, int((len(data) - tau)/out)):
    print(i)
    # current window
    window = np.array( data.iloc[i*out:i*out + tau, :] )
    # average out-of-sample variance associated with each lambda
    lmbd_variances = np.zeros((len(lmbd), 1))
    for fold in range(0, k):
        variances = []
        # sample values from in-sample data
        sample = np.random.choice(tau, out, replace=False)
        # remaining in-sample data
        mod_window = np.delete(window, sample, axis=0)
        # out-of-sample data
        outer = window[sample, :]
        # Estimated covariance matrix
        est_cov = np.cov(mod_window.T)
        ## CROSS-VALIDATION STEP
        for l in lmbd:
            # Portfolio weights
            mvp = minimize(Lq, x0, (est_cov, l), constraints=cons, tol=10**(-4)).x
            # out-of-sample variance associated to each lambda
            var_out = np.var(np.dot(mvp.T, outer.T).T )
            # append variance
            variances.append( var_out )
        variances = np.array(variances)
        variances.shape = (len(lmbd), 1)
        # update each lambda's corresponding variance
        lmbd_variances += variances/k
    # index of lambda*
    star = lmbd_variances.tolist().index(min(lmbd_variances)) 
    # lambda*
    lambda_star = lmbd[lmbd_variances.tolist().index(min(lmbd_variances))]
    ## END OF CROSS VALIDATION STEP
    # estimated covariance matrix
    est_cov = np.cov(window.T)
    # Portfolio weights
    mvp = minimize(Lq, x0, (est_cov, lambda_star), constraints=cons).x
    # out-of-sample data
    out_sample = np.array( data.iloc[i*out+tau:i*out+tau+out, :].T )
    # out-of-sample returns
    out_returns = np.dot(mvp.T, out_sample)
    lq_returns += out_returns.T.tolist()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [20]:
# compute Lq variance
lq_variance = np.var(lq_returns)
# compute Lq VaR at (1-var_thresh)% level
lq_var = np.percentile(lq_returns, 100-var_thresh)
# compute Lq Sharpe ratio
lq_sharpe = np.mean( lq_returns )/np.sqrt(lq_variance)
print(lq_variance)
print(lq_var)
print(lq_sharpe)

3.3816475904986015e-05
-0.008780028134422914
0.1583823620668408


In [21]:
variances = [equal_variance, mvp_variance, LASSO_variance, RIDGE_variance, W8_variance, SCAD_variance, log_variance, lq_variance]
v_risk = [equal_var, mvp_var, LASSO_var, RIDGE_var, W8_var, SCAD_var, log_var, lq_var]
sharpes = [equal_sharpe, mvp_sharpe, LASSO_sharpe, RIDGE_sharpe, W8_sharpe, SCAD_sharpe, log_sharpe, lq_sharpe]
penalties = ['Eq. weight','Unreg. MVP','LASSO', 'Ridge', '$w8LASS$', 'SCAD', 'Log', '$L_q$']